## **IMPORT LIBRARY**

In [2]:
import wfdb as wf
import neurokit2 as ECG
from scipy import signal as Sig
import numpy as np
from numpy.polynomial import chebyshev
import os

# Directory Path Saving
os.chdir('..')
dir = os.getcwd()
print(dir)

/home/senume/Project/Research/Person-Identification-using-Chebyshev-polynomial


## **CATEGORIES HEALTHY PATIENTS**

In [11]:
# Specifying Dataset Directory
os.chdir(dir + '/Dataset')
Patients_List = os.listdir()
os.chdir(dir)


# Healthy control List
HC = []

for patient in Patients_List:
    os.chdir(dir + '/Dataset/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')] 
    List = []
    for Audio in F:
        _, fields = wf.rdsamp( dir + '/Dataset/' + patient +'/' + Audio.replace('.dat',''))
        if 'Healthy control' in fields['comments'][4]:
            List.append(Audio.replace('.dat',''))
    if 'Healthy control' in fields['comments'][4]:
        HC.append([patient, List])


## **FUNCTION DEFINITION**

In [18]:
def Feature_Extraction(Patient, Audio, N, SR, lead, Peak_type, file):
    signal_array, fields = wf.rdsamp(f'/home/senume/Project/Research/Person-Identification-using-Chebyshev-polynomial/Dataset/{Patient}/{Audio}')
    signal = signal_array[:,lead]

    signal = ECG.ecg_clean(signal, sampling_rate = SR, method='pantompkins1985')

    _,R_Peaks = ECG.ecg_peaks(signal, Sampling_rate = SR, method='pantompkins1985')
    _,Peaks = ECG.ecg_delineate(signal, R_Peaks, sampling_rate = SR, show_type= 'all')

    Peaks.update(R_Peaks)

    Selected_Peaks = Peaks[f"ECG_{Peak_type}_Peaks"]

    for i in range(1, len(Selected_Peaks)):
        if isinstance(Selected_Peaks[i], np.int64) and isinstance(Selected_Peaks[i], np.int64):
            Beat = signal[Selected_Peaks[i-1]:Selected_Peaks[i]]
            t = np.linspace(-1,1,Beat.shape[0])
            Coeff = chebyshev.chebfit(t,Beat,(N-1))
            Feature = ''
            for element in Coeff:
                Feature += ',' + str(element)
            
            file.write(Audio + f'_{str(i-1)}' + Feature + '\n')


In [19]:
Degree = 50
Peak = 'S'  #P,Q,R,S,T

#os.mkdir(dir + f'/Feature_Dataset/{Peak}{Peak}_Interval')
os.mkdir(dir + f'/Feature_Dataset/{Peak}{Peak}_Interval/{str(Degree)}')
for Person in HC:
    
    file = open( dir + f'/Feature_Dataset/{Peak}{Peak}_Interval/{str(Degree)}/Feature_Extraction_{Peak}{Peak}_Interval_{Degree}_{Person[0]}.csv', 'a')

    file.write('ID')
    for k in range(1,Degree+1):
        file.write(f',CC_{k}')
    file.write('\n')

    for Audio in Person[1]:
        Feature_Extraction(Patient=Person[0], Audio = Audio, N = Degree, SR= 1000, lead = 11, Peak_type=Peak, file = file)
    file.close()
    break


/home/senume/anaconda3/envs/Person_identification_Env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/senume/anaconda3/envs/Person_identification_Env/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## **TESTING AREA**

In [101]:
SR = 1000
lead = 11
N = 5
Peak_type = 'S'


signal_array, fields = wf.rdsamp('/home/senume/Project/Research/Person-Identification-using-Chebyshev-polynomial/Dataset/patient001/s0010_re')
signal = signal_array[:,lead]

signal = ECG.ecg_clean(signal, sampling_rate = SR, method='pantompkins1985')

_,R_Peaks = ECG.ecg_peaks(signal, Sampling_rate = SR, method='pantompkins1985')
_,Peaks = ECG.ecg_delineate(signal, R_Peaks, sampling_rate = SR, show_type= 'all')

Peaks.update(R_Peaks)

Selected_Peaks = Peaks[f"ECG_{Peak_type}_Peaks"]

#file = open(dir + "/Feature_Dataset/" + str(N) + "/" + f"Feature_Extraction_{Patient}")

for i in range(1, len(Selected_Peaks)):
    if Selected_Peaks[i] != np.nan and Selected_Peaks[i-1] != np.nan:
        Beat = signal[Selected_Peaks[i-1]:Selected_Peaks[i]]
        t = np.linspace(-1,1,Beat.shape[0])
        Coeff = chebyshev.chebfit(t,Beat,N)

print(Coeff)




[-0.00567613 -0.01243997 -0.01650993 -0.00423749 -0.00644909  0.02581128]


In [7]:
Peak = 'S'

os.mkdir(dir + f'/Feature_Dataset/{Peak}{Peak}_Interval')